In [1]:
import pyparsing as pp
import re
import string
import nltk
from pyparsing import Word, alphas

In [21]:
def get_lines(filepath='convo_1.txt'):
    with open(filepath, 'r') as file:
        convo = file.read()
        convo_lines = [i for i in convo.split('\n') if len(i) > 0]
    return convo_lines

In [22]:
ls = get_lines()

In [23]:
ls

['BARTENDER',
 'Many bargains at the Old China Hand. Everything eight credits.',
 'JC DENTON',
 'Heard the name Tracer Tong?',
 'BARTENDER',
 'Tong, yes... friend of the Luminous Path Triad. And, by extension, enemy of the',
 'Red Arrow.',
 'JC DENTON',
 "What's his reputation... among the locals?",
 'BARTENDER',
 'Shrewd, but fair. Very well-connected. You want to buy... biomechanical',
 'augmentation?',
 'JC DENTON',
 'Just curious.',
 'Is there a lot of crime in Hong Kong?',
 'BARTENDER',
 "Much more than before, yes. Now it's very easy for the smugglers.",
 'JC DENTON',
 'What kind of smugglers?',
 'BARTENDER',
 'Slaves, heroin, biotech...',
 'JC DENTON',
 'Is there really a slave-trade?',
 'BARTENDER',
 'Oh, sure. Visit the Lucky Money underneath Wan Chai, the Red Arrow',
 'headquarters... All of their girls, they are kidnapped in Thailand or the',
 'Philippines. Not a pretty business.',
 'JC DENTON',
 'What sort of biotech smuggling goes on?',
 'BARTENDER',
 'Mostly VersaLife, th

In [17]:
[i for i in convo.split('\n') if len(i) > 0]

['BARTENDER',
 'Many bargains at the Old China Hand. Everything eight credits.',
 'JC DENTON',
 'Heard the name Tracer Tong?',
 'BARTENDER',
 'Tong, yes... friend of the Luminous Path Triad. And, by extension, enemy of the',
 'Red Arrow.',
 'JC DENTON',
 "What's his reputation... among the locals?",
 'BARTENDER',
 'Shrewd, but fair. Very well-connected. You want to buy... biomechanical',
 'augmentation?',
 'JC DENTON',
 'Just curious.',
 'Is there a lot of crime in Hong Kong?',
 'BARTENDER',
 "Much more than before, yes. Now it's very easy for the smugglers.",
 'JC DENTON',
 'What kind of smugglers?',
 'BARTENDER',
 'Slaves, heroin, biotech...',
 'JC DENTON',
 'Is there really a slave-trade?',
 'BARTENDER',
 'Oh, sure. Visit the Lucky Money underneath Wan Chai, the Red Arrow',
 'headquarters... All of their girls, they are kidnapped in Thailand or the',
 'Philippines. Not a pretty business.',
 'JC DENTON',
 'What sort of biotech smuggling goes on?',
 'BARTENDER',
 'Mostly VersaLife, th

In [8]:
# define grammar of a greeting
pp.ParserElement.setDefaultWhitespaceChars(' \t')
l = pp.OneOrMore(pp.Word(pp.alphas)) + pp.Literal('.')

l.parse_string(convo)

ParseResults(['Many', 'bargains', 'at', 'the', 'Old', 'China', 'Hand', '.'], {})

In [ ]:
    log = Word(printables)
    print(log)

    log1 = Word(alphanums+"-")
    print(log1)

    log_display = ("Starting Daily apt upgrade and clean 
    activities...")
    data = log.parseString(log_display)
    print(data.dump())
    print()